* https://www.kaggle.com/code/alexandervc/cafa5-esm2-protein-embeddings-visualizations
    * tsne, umap, pca etc' - cluster based on embeddings
* Length - is not an embedding (And different feature scale). Maybe drop it from drawing embeddings? (could use for explanaing, same as for mistake or virus or not)

In [16]:
!conda update --all -y

Channels:
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/hf


The following packages will be UPDATED:

  libgfortran                       5.0.0-13_2_0_h97931a8_3 --> 5.0.0-11_3_0_h97931a8_32 

The following packages will be DOWNGRADED:

  libblas                           3.9.0-21_osx64_openblas --> 3.9.0-17_osx64_openblas 
  libcblas                          3.9.0-21_osx64_openblas --> 3.9.0-17_osx64_openblas 
  libgfortran5                            13.2.0-h2873a65_3 --> 12.2.0-he409387_32 
  liblapack                         3.9.0-21_osx64_openblas --> 3.9.0-17_osx64_openblas 
  libopenblas                      0.3.26-openmp_hfef2a42_0 --> 0.3.23-openmp_h429af6e_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [17]:
import pandas as pd
import numpy as np
import random

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing  import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA


marker1 =  'o' # 'o' #
palette1 = 'rainbow' #  "viridis"  #  'rainbow' #  "viridis"# 
alpha1 = 1 
str_data_inf = 'Human - Virus Embeddings'

In [18]:
FAST_RUN = False#True

In [19]:
df_extra = pd.read_csv("./outputs/mistake_preds_4cv_650+len_lr_2024.csv.gz",usecols=['Sequence',"virus",
#         'model_pred_proba_vir',
#        'model_pred_vir',
       'mistake', 'Baltimore', 'Family', 'Genome Composition',  'Protein names', 
       'UR50_Cluster ID',  'human_host'])
# df_extra["Cluster name"] = X['UR50_Cluster ID'].str.replace("Cluster: ","")
df_extra = df_extra.sort_values("Sequence").rename(columns={"Protein names":"Cluster name"}).set_index(["Cluster name"])
df_extra

,Sequence,virus,mistake,Baltimore,Family,Genome Composition,UR50_Cluster ID,human_host
Cluster name,,,,,,,,
Tyrosine-protein kinase transforming protein Fes (EC 2.7.10.2),AARADGTMGFSSELCSPQGHGAEQQMQEAELRLLEGMRKWMAQRVK...,1,True,ssRNA-RT,Retroviridae,ssRNA-RT,P00542,False
Uncharacterized protein UL81,ACSTPHYDLPLICAATWTATCVPRQTSAARCGPCPRHDPHPSAPLP...,1,False,dsDNA,Herpesviridae,dsDNA,P16825,True
Structural polyprotein [Cleaved into: Protein VP1 (Virion protein 2); Protein VP4 (Virion protein 4); Protein VP2 (Virion protein 3); Protein VP3 (Virion protein 1)],ADQETNTSNVHNTQLASTSEENSVETEQITTFHDVETPNRINTPMA...,1,False,ssRNA,Dicistroviridae,ssRNA(+),Q9DSN8,False
Tyrosine-protein kinase transforming protein SEA (EC 2.7.10.2),ADSPGLARPHAHFASAGADAAGGGSPVLLLRTTSCCLEDLRPELLE...,1,False,ssRNA-RT,Retroviridae,ssRNA-RT,P23049,False
Uncharacterized protein UL62,AEKSSSKTSQKKAQKSYRAIFLPAWSKNAPRLSTPFSLVRHVGAQN...,1,False,dsDNA,Herpesviridae,dsDNA,P16819,True
...,...,...,...,...,...,...,...,...
Endogenous retrovirus group K member 10 Pro protein (HERV-K10 Pro protein) (HERV-K107 Pro protein) (HERV-K_5q33.3 provirus ancestral Pro protein) (EC 3.4.23.50) (Protease) (Proteinase) (PR),WASQVSENRPVCKAIIQGKQFEGLVDTGADVSIIALNQWPKNWPKQ...,0,False,NaN,NaN,NaN,P10265,NaN
Endogenous retrovirus group K member 24 Pro protein (HERV-K101 envelope protein) (HERV-K_22q11.21 provirus ancestral Pro protein) (EC 3.4.23.50) (Protease) (Proteinase) (PR),WASQVSENRPVCKAIIQGKQFEGLVDTGADVSIIALNQWPKNWPKQ...,0,False,NaN,NaN,NaN,P63129,NaN
Endogenous retrovirus group K member 6 Pro protein (HERV-K(C7) Pro protein) (HERV-K(HML-2.HOM) Pro protein) (HERV-K108 Pro protein) (HERV-K_7p22.1 provirus ancestral Pro protein) (EC 3.4.23.50) (Protease) (Proteinase) (PR),WASQVSENRPVCKAIIQGKQFEGLVDTGADVSIIALNQWPKNWPKQ...,0,False,NaN,NaN,NaN,Q9Y6I0,NaN


In [20]:
X = pd.concat([pd.read_csv("hum_vir_swp-globalEmbed-train.csv.gz",nrows= 300 if FAST_RUN else 1e9),
               pd.read_csv("hum_vir_swp-globalEmbed-test.csv.gz",nrows= 200 if FAST_RUN else 1e9)
              ])
v_lengths = X["Length"]
X.drop(columns=['Mass', "av_mw","Length"],errors="ignore",inplace=True)
X["Cluster name"] = X["Cluster name"].str.replace("Cluster: ","")
groups = X["Cluster name"].str.replace("Cluster: ","") # "Entry"
y = X.pop("virus")
X.sort_values("Sequence",inplace=True)

###
df_extra = df_extra.loc[df_extra["Sequence"].isin(X["Sequence"])]
print(df_extra.shape,"df_extra filtered by seq shape")

X = X.set_index(["Cluster name"]).select_dtypes("number")
X

(25117, 8) df_extra filtered by seq shape


,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1014,global_embed_1015,global_embed_1016,global_embed_1017,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023
Cluster name,,,,,,,,,,,,,,,,,,,,,
Tyrosine-protein kinase transforming protein Fes,-0.005090,0.045840,0.036000,-0.001109,-0.02153,0.03790,-0.010110,-0.02441,0.03024,0.005013,...,-0.015950,-0.022610,-0.028320,-0.003395,0.023440,-0.003117,-0.018250,-0.01100,-0.022920,0.015690
Uncharacterized protein UL81,0.054930,0.103200,0.025900,0.031170,-0.11084,-0.06350,-0.031280,-0.13670,-0.01321,-0.068660,...,0.022660,-0.005030,0.055850,-0.174800,0.001076,0.076540,0.017070,0.08340,0.007790,-0.044000
Structural polyprotein,-0.047760,-0.069600,0.007668,0.031950,0.02120,0.02919,0.049470,-0.04570,-0.04944,0.010130,...,-0.108900,-0.005657,0.016340,0.040200,0.052220,0.084050,0.003914,0.06125,0.121000,0.029560
Tyrosine-protein kinase transforming protein SEA,0.001788,0.054720,0.037660,-0.007744,-0.02728,0.02048,-0.011955,-0.04916,0.02608,-0.006973,...,-0.000979,-0.002686,-0.005190,-0.037840,0.017170,-0.003496,-0.011210,-0.00557,-0.009470,0.022060
Uncharacterized protein UL62,-0.019030,0.098800,0.004060,0.009590,-0.02467,-0.06620,-0.004063,-0.07190,0.01154,-0.109250,...,0.084800,-0.050380,-0.001919,-0.097530,-0.064200,0.119700,0.057400,0.10690,0.055150,0.015730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Endogenous retrovirus group K member 10 Pro protein,0.010620,-0.010315,0.058440,0.001824,-0.03162,0.03009,0.030400,-0.08550,-0.05576,0.001459,...,0.044560,-0.045780,0.073600,-0.148600,0.028410,-0.037540,-0.052640,0.03397,-0.018600,-0.042140
Endogenous retrovirus group K member 24 Pro protein,0.006430,-0.008064,0.056920,0.003757,-0.03455,0.03314,0.024020,-0.08484,-0.05480,-0.003630,...,0.042600,-0.042400,0.077100,-0.150500,0.033260,-0.037350,-0.058900,0.03265,-0.018700,-0.043100
Endogenous retrovirus group K member 6 Pro protein,0.008290,-0.009820,0.054930,0.001905,-0.03394,0.02620,0.027910,-0.08300,-0.05396,0.001145,...,0.049500,-0.045750,0.078800,-0.150100,0.027330,-0.042500,-0.053960,0.03445,-0.016530,-0.041750


In [21]:
def get_joint_label(row):
    gt_label = "Virus" if row["virus"] else "Human"
    gt_mistake = "Mistake" if row["mistake"] else "Correct" 
    return gt_label+", "+gt_mistake

In [22]:
# joint_label = "mistake:"+df_extra["mistake"].astype(str).values +",Virus:"+  df_extra["virus"].astype(str).values#y.astype(str)

joint_label =df_extra.apply(get_joint_label,axis=1)
joint_label.values

array(['Virus, Mistake', 'Virus, Correct', 'Virus, Correct', ...,
       'Human, Correct', 'Human, Correct', 'Virus, Mistake'], dtype=object)

* Embeddings values range from -.08 to +0.14. 
* standard scaler of length gives  -1.3 to + 3.37 - large values 

In [23]:
if not FAST_RUN:
    assert df_extra.shape[0] == X.shape[0]

In [24]:
X.max().describe().round(2)

count    1024.00
mean        0.22
std         0.07
min        -0.08
25%         0.17
50%         0.21
75%         0.25
max         0.69
dtype: float64

In [25]:
# # scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = StandardScaler()
# X["Length"] = scaler.fit_transform(X[["Length"]])
# X["Length"].describe().round(2)

In [26]:
X = X.values

In [27]:
%%time

pca_big = PCA(128) # n_components='mle') # mle = very slow/ stuck?
pca = PCA(n_components=6)
r = pca.fit_transform(X)

print(np.sum(pca.explained_variance_ratio_))

0.3360261636905406
CPU times: user 6.22 s, sys: 3.15 s, total: 9.37 s
Wall time: 828 ms


In [28]:
%%time
X_pca = pca_big.fit_transform(X)
print(np.sum(pca_big.explained_variance_ratio_))
print(X_pca.shape)

0.838080120473659
(25117, 128)
CPU times: user 11.6 s, sys: 4.92 s, total: 16.6 s
Wall time: 1.29 s


In [29]:
v4color = pd.Series(r[:,0])
v4color.name = 'PCA1'

### TSNE
* https://www.kaggle.com/code/alexandervc/cafa5-esm2-protein-embeddings-visualizations#OpenTSNE

In [ ]:
!pip install opentsne
from openTSNE import TSNE

In [ ]:
%%time
# if str_data_inf != 'Medulloblastoma Intergrated GSE124814': # Fail in that case by unclear reason 

reducer = TSNE(
#     perplexity=30,
#     perplexity=15,
    metric= "cosine", #"euclidean",
    n_jobs=-1,
    random_state=42,
#     verbose=True,
)
r = reducer.fit(X)

for (i,j) in [(0,1)]:#,(0,2),(1,2),(2,3),(2,4),(3,4)]:
    fig = plt.figure(figsize = (20,12) ); c = 0
    ax = sns.scatterplot(x = r[:,i],y=r[:,j], hue = v4color,
                         palette = palette1 , marker =  marker1 , alpha = alpha1 )
    if 1:
        plt.setp(ax.get_legend().get_texts(), fontsize=12) # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize=12) # for legend title        
    plt.title(' openTSNE '+ str_data_inf + ' n_samples='+str(len(r)), fontsize = 20)
    plt.show()

In [ ]:
for (i,j) in [(0,1)]:#,(0,2),(1,2),(2,3),(2,4),(3,4)]:
    fig = plt.figure(figsize = (20,12) ); c = 0
    ax = sns.scatterplot(x = r[:,i],y=r[:,j], hue = y,
                         palette = palette1 , marker =  marker1 , alpha = alpha1 )
    if 1:
        plt.setp(ax.get_legend().get_texts(), fontsize=12) # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize=12) # for legend title        
    plt.title(' openTSNE '+ str_data_inf + ' n_samples='+str(len(r)), fontsize = 20)
    plt.show()

In [ ]:
for (i,j) in [(0,1)]:#,(0,2),(1,2),(2,3),(2,4),(3,4)]:
    fig = plt.figure(figsize = (20,12),dpi=600 ); c = 0
    ax = sns.scatterplot(x = r[:,i],y=r[:,j], hue = joint_label#df_extra["mistake"],
                         ,palette = palette1 , marker =  marker1 , alpha = alpha1 )
    if 1:
        plt.setp(ax.get_legend().get_texts(), fontsize=18) # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize=18) # for legend title        
    # plt.title('TSNE '+ str_data_inf + ' n_samples='+str(len(r)), fontsize = 20)
    plt.title('TSNE '+ str_data_inf, fontsize = 20)
    plt.show()

## UMAP

In [ ]:
# !pip install umap-learn  opentsne
# # ncvis

# !pip install opentsne

In [ ]:
a

```
pip uninstall umap
pip install umap-learn
```


In [ ]:
# !conda uninstall umap-learn -y
# !pip install umap-learn opentsne

In [ ]:
# !pip install umap-learn opentsne

# !conda install -c conda-forge umap-learn -y
# !conda install  umap-learn -y

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

import umap 
# import umap.umap_ as umap

r = umap.UMAP(metric="cosine").fit_transform(X)

# v4color = df_corpus['sequence'].apply(lambda x: len(x))
# v4color.name = 'sequence length'
# v4color = v_lengths
# v4color.name = 'sequence length'

print(r.shape)

for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    fig = plt.figure(figsize = (20,12),dpi=600 )
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = joint_label,
                    palette = 'rainbow', markers=["v", "v", "h","h"] )
    plt.xlabel('UMAP'+str(i+1), fontsize = 16)
    plt.ylabel('UMAP'+str(j+1), fontsize = 16)
    plt.show()

In [ ]:
y

In [ ]:
for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    fig = plt.figure(figsize = (20,12),dpi=600 )
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = joint_label,
                    style=y, #"virus",
                    palette = 'rainbow' ) # , markers=["v", "V", "H","h"]
    plt.xlabel('UMAP'+str(i+1), fontsize = 16)
    plt.ylabel('UMAP'+str(j+1), fontsize = 16)
    plt.show()

In [ ]:
for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = y )
    plt.xlabel('UMAP'+str(i+1), fontsize = 20)
    plt.ylabel('UMAP'+str(j+1), fontsize = 20)
    plt.show()

In [ ]:
# ## plot by mistake # needto import that + align first
# for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
#     fig = plt.figure(figsize = (20,12),dpi=600 )
#     sns.scatterplot(x = r[:,i], y = r[:,j], hue = df_extra["mistake"] )
#     plt.xlabel('UMAP'+str(i+1), fontsize = 20)
#     plt.ylabel('UMAP'+str(j+1), fontsize = 20)
#     plt.show()

### supervised umap
* https://umap-learn.readthedocs.io/en/latest/supervised.html

In [ ]:
%%time

r = umap.UMAP().fit_transform(X, y=y)

# v4color = df_corpus['sequence'].apply(lambda x: len(x))
# v4color.name = 'sequence length'
# v4color = v_lengths
# v4color.name = 'sequence length'

print(r.shape)

for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = joint_label,
#                     palette = 'rainbow'
                   )
    plt.xlabel('UMAP'+str(i+1), fontsize = 16)
    plt.ylabel('UMAP'+str(j+1), fontsize = 16)
    plt.show()

In [ ]:
print("using PCA base")

r = umap.UMAP().fit_transform(X_pca, y=y)

print(r.shape)

for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = joint_label,
#                     palette = 'rainbow'
                   )
    plt.xlabel('UMAP'+str(i+1), fontsize = 16)
    plt.ylabel('UMAP'+str(j+1), fontsize = 16)
    plt.show()

In [ ]:
for i,j in [(0,1)]: # ,(0,2),(0,3),(1,2),(1,3),(2,3)]:
    sns.scatterplot(x = r[:,i], y = r[:,j], hue = joint_label,
                    palette = 'rainbow'
                   )
    plt.xlabel('UMAP'+str(i+1), fontsize = 16)
    plt.ylabel('UMAP'+str(j+1), fontsize = 16)
    plt.show()